In [ ]:
pip install azure-storage-blob

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.2/394.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-2a22fcdc-f424-4c46-bfea-605ed84759c6
    Can't uninstall 'typing_extensions'. No files were found to uninstall.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [ ]:
dbutils.library.restartPython()

In [ ]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

In [ ]:
# checking that data exists or not into storage
def view_blob(container_name, blob_name, connection_string):
    # connection_string = f"DefaultEndpointsProtocol=https;AccountName={storage_account_name};AccountKey={storage_account_key};EndpointSuffix=core.windows.net"
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(blob_name)

    try:
        blob_data = blob_client.download_blob().readall()
        print(f"Contents of blob '{blob_name}':\n")
        print(blob_data.decode("utf-8"))
    except Exception as e:
        print(f"Error accessing blob '{blob_name}': {str(e)}")

# Replace the following with your actual values
storage_account_name = "your_storage_account_name"
storage_account_key = "your_storage_account_key"
container_name = "bigdatacontainer1"
blob_name = "blobtable"
shared_access_key = "add-key-here"
connection_string = f"DefaultEndpointsProtocol=https;AccountName=bigdatastorage30;AccountKey={shared_access_key};EndpointSuffix=core.windows.net"

# view_blob(container_name, blob_name, connection_string)


In [ ]:
# def list_blobs(container_name, account_key):
#     account_name = "bigdatastorage30"

#     blob_service_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net", credential=account_key)
#     container_client = blob_service_client.get_container_client(container_name)

#     print(f"Blobs in container '{container_name}':")
#     blobs = container_client.list_blobs()
#     for blob in blobs:
# print(blob.name)

  File <command-3614457368864538>, line 10
    print(blob.name)
    ^
IndentationError: unexpected indent


In [ ]:
# list_blobs(container_name, shared_access_key)

In [ ]:
# reading the data and storing into spark data frame
container_name = "bigdatacontainer10"
delta_table_name = "deltatable"  # replace with your desired table name
fully_qualified_account_name = "bigdatastorage30.blob.core.windows.net"

shared_access_key = "add-key-here"
connection_string = f"DefaultEndpointsProtocol=https;AccountName=bigdatastorage30;AccountKey={shared_access_key};EndpointSuffix=core.windows.net"

# spark.conf.set("fs.azure.account.key.{fully_qualified_account_name}", connection_string)
spark.conf.set(f"fs.azure.account.key.{fully_qualified_account_name}", shared_access_key)

delta_table_path = f"wasbs://{container_name}@{fully_qualified_account_name}/{delta_table_name}"
delta_df = spark.read.format("parquet").load(delta_table_path)

delta_df.createOrReplaceTempView("deltatable")
result = spark.sql("SELECT * FROM deltatable")

result.show()

+------------------+------------------+------------------+------------------+------------------+-------+------+----------+
|              open|              high|               low|             close|          adjclose| volume|ticker|      date|
+------------------+------------------+------------------+------------------+------------------+-------+------+----------+
| 56.33047103881836| 56.46459197998047|    48.19384765625| 51.50214767456055| 43.75774383544922|4674353|     A|2000-01-03|
| 48.73032760620117| 49.26681137084961|46.316165924072266|47.567955017089844|40.415138244628906|4765083|     A|2000-01-04|
| 47.38912582397461|47.567955017089844|43.141990661621094|  44.6173095703125| 37.90816879272461|5758642|     A|2000-01-05|
| 44.08082962036133| 44.34907150268555| 41.57725143432617|42.918453216552734| 36.46477508544922|2534434|     A|2000-01-06|
| 42.24785232543945|47.165592193603516|42.203147888183594|46.494991302490234| 39.50350570678711|2819626|     A|2000-01-07|
|49.356224060058

In [ ]:
temp_df = delta_df.toPandas()
print(temp_df.duplicated().sum())

0


open        0
high        0
low         0
close       0
adjclose    0
volume      0
ticker      0
date        0
dtype: int64

In [ ]:
# r = spark.sql("SELECT * FROM deltatable ORDER BY date DESC LIMIT 20")
# r.show()

+------------------+------------------+------------------+------------------+------------------+--------+------+----------+
|              open|              high|               low|             close|          adjclose|  volume|ticker|      date|
+------------------+------------------+------------------+------------------+------------------+--------+------+----------+
| 189.3699951171875| 189.9199981689453|186.61000061035156| 188.5399932861328| 187.0249786376953|  353200|   AJG|2022-12-30|
|45.459999084472656|46.689998626708984| 45.41999816894531| 46.68000030517578|45.554656982421875| 4752600|   APA|2022-12-30|
| 83.93000030517578| 84.88999938964844| 83.61000061035156| 84.30000305175781| 84.30000305175781|  675000|  AKAM|2022-12-30|
|149.75999450683594|149.85000610351562| 147.8300018310547|149.64999389648438|148.82179260253906|  699800|     A|2022-12-30|
|215.13999938964844|218.47999572753906|213.60000610351562|216.86000061035156| 215.6053924560547|  951800|   ALB|2022-12-30|
|128.410

In [ ]:
# AAP = spark.sql("SELECT * FROM deltatable WHERE ticker = 'AAPL'").toPandas()
# ax = AAP['adjclose'].plot(legend=True, figsize=(12,5))
# ax.set_xticks(range(len(AAP['date'])))
# ax.set_xticklabels(AAP['date'].strftime('%Y-%m-%d'), rotation=45, ha='right')

# plt.show()


In [ ]:
# calculate the average closing price by ticker for a specific year
q1 = spark.sql("SELECT ticker, YEAR(date) AS year, AVG(adjclose) AS avg_adjusted_close_price FROM deltatable WHERE YEAR(date) = 2022 GROUP BY ticker, YEAR(date) ORDER BY ticker, year")

q1.show()

+------+----+------------------------+
|ticker|year|avg_adjusted_close_price|
+------+----+------------------------+
|     A|2022|      131.68152335725458|
|   AAL|2022|      15.269920288329105|
|  AAPL|2022|      153.52379571679103|
|  ABBV|2022|      140.75537136731396|
|  ABNB|2022|      125.09908398024115|
|   ABT|2022|      108.82109054626221|
|  ACGL|2022|       48.46633481029495|
|   ACN|2022|      294.02998820149094|
|  ADBE|2022|       396.1215133363032|
|   ADI|2022|      154.64466487079028|
|   ADM|2022|       82.36897979979496|
|   ADP|2022|      222.79854586779834|
|  ADSK|2022|      206.29179309373833|
|   AEE|2022|       85.58245533491036|
|   AEP|2022|       89.91474826402398|
|   AES|2022|      22.770140112158785|
|   AFL|2022|       59.69602238324534|
|   AIG|2022|       55.39151315575102|
|   AIZ|2022|      155.42408086769134|
|   AJG|2022|       170.1944014104714|
+------+----+------------------------+
only showing top 20 rows



In [ ]:
# find the top 10 stocks with the highest average volume between a particular year range
q2 = spark.sql("SELECT ticker, AVG(volume) AS avg_volume FROM deltatable WHERE YEAR(date) BETWEEN 2015 AND 2017 GROUP BY ticker ORDER BY avg_volume DESC LIMIT 10")

q2.show()

+------+--------------------+
|ticker|          avg_volume|
+------+--------------------+
|  AAPL|1.5660558516556293E8|
|   BAC| 9.058063801324503E7|
|  TSLA| 7.630352781456953E7|
|  AMZN| 7.625334304635762E7|
|  NVDA| 4.793889218543047E7|
| GOOGL| 3.838144105960265E7|
|   AMD|3.6463615894039735E7|
|  GOOG| 3.586324285033113E7|
|     T| 3.407514864503311E7|
|     F| 3.348954847682119E7|
+------+--------------------+



In [ ]:
# Find Stocks with Largest Daily Price Fluctuations for particular year with date

q3 = spark.sql("SELECT ticker, MAX(high - low) AS largest_daily_fluctuation, MAX(CASE WHEN high - low = (SELECT MAX(high - low) FROM deltatable WHERE ticker = outer_tbl.ticker AND YEAR(date) = 2021) THEN date END) AS date_of_max_fluctuation FROM deltatable AS outer_tbl WHERE YEAR(date) = 2021 GROUP BY ticker ORDER BY largest_daily_fluctuation DESC")

q3.show()

+------+-------------------------+-----------------------+
|ticker|largest_daily_fluctuation|date_of_max_fluctuation|
+------+-------------------------+-----------------------+
|   NVR|          265.52001953125|             2021-03-05|
|  BIIB|       182.54998779296875|             2021-06-07|
|  BKNG|          177.47998046875|             2021-02-25|
|   AZO|        137.9500732421875|             2021-12-07|
|   CMG|        121.4400634765625|             2021-07-21|
|  ILMN|       103.44003295898438|             2021-02-12|
|   MTD|         84.8299560546875|             2021-09-27|
|  MRNA|        84.09698486328125|             2021-08-09|
|  GNRC|        69.20999145507812|             2021-11-02|
|  MKTX|          66.469970703125|             2021-03-03|
|   DPZ|          64.719970703125|             2021-07-22|
|   BIO|        61.60003662109375|             2021-11-05|
|  EPAM|        61.38995361328125|             2021-12-08|
|   NOW|         58.6199951171875|             2021-12-0

In [ ]:
# Discover Days with Unusually High Trading Volume (Market crash)
q4 = spark.sql("SELECT date, SUM(volume) AS total_volume FROM deltatable GROUP BY date ORDER BY total_volume DESC LIMIT 10")

q4.show()

+----------+------------+
|      date|total_volume|
+----------+------------+
|2008-10-10| 11077903825|
|2008-01-23| 10217809448|
|2008-10-08|  9381945565|
|2008-10-16|  9147144682|
|2008-09-18|  9137067270|
|2011-08-08|  8858787082|
|2000-09-29|  8784221493|
|2008-11-21|  8781691206|
|2008-11-20|  8722759911|
|2008-10-06|  8581705442|
+----------+------------+



In [ ]:
# Calculate Yearly Average Adjusted Close Price for particular Ticker
q5 = spark.sql("SELECT ticker, YEAR(date) AS year, AVG(adjclose) AS avg_yearly_adj_close FROM deltatable WHERE ticker = 'TSLA' GROUP BY ticker, YEAR (date) ORDER BY year")

q5.show()

+------+----+--------------------+
|ticker|year|avg_yearly_adj_close|
+------+----+--------------------+
|  TSLA|2010|  1.5561230677824753|
|  TSLA|2011|  1.7869841338150085|
|  TSLA|2012|  2.0779066395759584|
|  TSLA|2013|   6.960081993587433|
|  TSLA|2014|  14.888605814131479|
|  TSLA|2015|  15.336193088501219|
|  TSLA|2016|  13.984484154080588|
|  TSLA|2017|   20.95441962618277|
|  TSLA|2018|   21.15399465522918|
|  TSLA|2019|   18.23534659733848|
|  TSLA|2020|   96.66568880495818|
|  TSLA|2021|   259.9981617397732|
|  TSLA|2022|   263.0930809841688|
+------+----+--------------------+

